In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
from tqdm import tqdm
import requests

In [2]:
locator = Nominatim().geocode

In [3]:
data = pd.read_csv("~/Menu.csv")
places = [place for place in set(data["place"]) if str(place) != 'nan']
# places

In [4]:
def get_location(address):
    try:
        return locator(address,timeout=10)
    except:
        return get_location(address)

dirt = ['!','?',':',',','.','[',']','(',')',';']
    
def clean_address(address):
    for d in dirt:
        address = address.replace(d," ")
    return " ".join(address.split())

def get_location_google(address,t=0):
    try:
        url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address.replace(" ","+"))
        response = requests.get(url)
        
        if response.json()['status']=="OK":
            return response.json()['results'][0]['geometry']['location']
        
        elif response.json()['status']=="ZERO_RESULTS":
            return None
        
        else:
            t+=1
            if t<10:
                return get_location_google(address,t)
            else:
                return None
    except:
        t+=1
        if t<10:
            return get_location_google(address,t)
        else:
            return None

In [5]:
located = []
unlocated = []

l = 0
u = 0

for place in tqdm(places):
    clean_place = clean_address(place)
    location = get_location(clean_place)

    if location:
        l+=1
        located.append((place,{'lat':location.latitude,'lng':location.longitude}))
        
    else:
        location = get_location_google(clean_place)
        
        if location:
            l+=1
            located.append((place,location))
        else:
            u+=1
            unlocated.append(place)
            
    print("\rLocated: {} \t Unlocated: {}".format(l,u),end="")
    
# located = []
# with open("locations-2017-11-16 12:42:08.068035.txt") as f:
#     for line in f:
#         if len(line)>1:
#             located.append(line.split("->")[0][0:-1])
# unlocated = [place for place in places if place not in located]

print("Located: {}".format(len(located)))
print("Unlocated: {}".format(len(unlocated)))

Located: 2018
Unlocated: 1696


In [ ]:
from datetime import datetime
file = "locations-{}.txt".format(datetime.now())

with open(file,'w') as f:
    for place in located:
        f.write("{} -> {},{}\n".format(place[0],place[1].latitude,place[1].longitude))

with open("unlocateds.txt",'w') as f:
    for place in unlocated:
        f.write("{}\n".format(place))